In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from common import *
from models import LogReg, NeuralNetwork, Layer

import seaborn as sns
from matplotlib.pyplot import figure

import numpy as np 
%matplotlib inline

In [ ]:
cancer = load_breast_cancer()

X_train, X_test, y_train, y_test = train_test_split(cancer.data,cancer.target,test_size=0.25,random_state=4155)
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:

eta_list = np.linspace(0.0001,0.1,10)
lmb_list = np.linspace(0,.9,10)
acu_scores = np.zeros(len(eta_list))
 
heatmap_mtrx = np.zeros(shape=(len(eta_list), len(lmb_list)))


for eta in range(len(eta_list)):
    for lmb in range(len(lmb_list)):
        logreg_SGD = LogReg(eta = eta_list[eta], lmb = lmb_list[lmb])
        logreg_SGD.fit(X_train_scaled,y_train,batch_size= 5, epochs=10, solver = "SGD")
    
        heatmap_mtrx[eta,lmb] =  logreg_SGD.accuracy(X_test_scaled, y_test)

print(f"last matr elem: {heatmap_mtrx[-1,-1]}")
figure(figsize=(12,10))
eta_list = np.around(eta_list, decimals=4)
lmb_list = np.around(lmb_list, decimals=4)
gridsearch = sns.heatmap(heatmap_mtrx,annot=True, xticklabels= eta_list, yticklabels= lmb_list)
gridsearch.invert_yaxis()
gridsearch.set_xticklabels(gridsearch.get_xticklabels(),rotation = 80)

plt.title("Eta, Lambda grid search")
plt.xlabel("Eta")
plt.ylabel("Lambda")



In [ ]:
logreg_NRM = NumpyLogReg()
logreg_NRM.fit(X_train_scaled,y_train,batch_size= 5, epochs=1, solver = "NRM")
logreg_NRM.accuracy(X_test_scaled, y_test)

In [ ]:
from sklearn.linear_model import LogisticRegression

def sci_kit_test_acu(X_train, X_test, y_train, y_test):
    logreg = LogisticRegression(penalty="l2", C = 0.2)
    logreg.fit(X_train, y_train)
    y_pred = logreg.predict(X_test)
    

    return sum(y_pred == y_test) / len(y_pred)

sci_kit_test_acu(X_train_scaled, X_test_scaled, y_train, y_test)



### Compare neural network classification results with Logistic regression results:


In [ ]:
# Hyper parameters
lr = 0.2
epochs = 200
batch_size = 10
hidden1_neurons = 50
input_dim = X_train.shape[1]

# Model architecture
hidden1 = Layer(input_dim, hidden1_neurons, activation="sigmoid", name="hidden1")
hidden2 = Layer(hidden1.neurons, hidden1.neurons*2, activation="sigmoid", name="hidden2")
hidden3 = Layer(hidden2.neurons, hidden2.neurons//2, activation="sigmoid", name="hidden3")
output = Layer(hidden3.neurons, nbf_neurons=1, activation="sigmoid" , name="output")

model = NeuralNetwork(learning_rate=lr, network_type="classification")
model.add(hidden1)
model.add(output)

model.fit(X_train_scaled, y_train.reshape(-1,1), batch_size=batch_size, epochs=epochs)
y_hat = model.predict(X_test_scaled)
y_hat = y_hat.flatten()

def accuracy(y_hat, y_true):
    return np.sum(y_hat == y_true) / y_true.shape[0]

print(accuracy(y_hat,y_test))

# With regularization

In [ ]:
# Hyper parameters
lmb = 0.0001

# Model architecture
hidden1 = Layer(input_dim, hidden1_neurons, activation="sigmoid", name="hidden1")
hidden2 = Layer(hidden1.neurons, hidden1.neurons*2, activation="sigmoid", name="hidden2")
hidden3 = Layer(hidden2.neurons, hidden2.neurons//2, activation="sigmoid", name="hidden3")
output = Layer(hidden3.neurons, nbf_neurons=1, activation="sigmoid" , name="output")

model = NeuralNetwork(learning_rate=lr , network_type="classification", lmb=lmb)
model.add(hidden1)
model.add(output)

#TODO: Had to reshape u_train(?)
model.fit(X_train_scaled, y_train.reshape(-1,1), batch_size=batch_size, epochs=epochs)
y_hat = model.predict(X_test_scaled)
y_hat = y_hat.flatten()

def accuracy(y_hat, y_true):
    return np.sum(y_hat == y_true) / y_true.shape[0]

print(accuracy(y_hat,y_test))